In [1]:
doccano_file_path = "/home/xiaojin/Code/DeepNER/data/crf_data/train-ann-0325.json1"
official_file_path = "/home/xiaojin/Code/DeepNER/data/crf_data/file_official.json1"
write_path = "/home/xiaojin/Code/DeepNER/data/crf_data/"

In [2]:
import jsonlines

In [ ]:
def convert_to_df(doccano_file_path, official=False, refine_file=False):
    with jsonlines.open(doccano_file_path) as f:
        """
        {"id": 1, "text": "EU rejects ...",
        "labels": [[0,2,"ORG"], [11,17, "MISC"], [34,41,"ORG"]]}
        """
        labeled_data = []
        for line in f:
            # print(line)
            text = line["text"]
            if official:
                labels = [[l[0], l[1], l[2]] for l in line["labels"]]
            elif refine_file:
                annotations = line["annotations"]
                label_maping = {
                    1: "Caps",
                    2: "LC",
                    3: "MVI",
                    4: "Sate",
                    5: "TNM",
                    6: "Tdiff",
                    7: "This",
                    8: "Tloc",
                    9: "Tnum",
                    10: "Tsize",
                    11: "Caps",
                    12: "LC",
                    13: "MVI",
                    14: "Sate",
                    15: "TNM",
                    16: "Tdiff",
                    17: "This",
                    18: "Tloc",
                    19: "Tnum",
                    20: "Tsize",
                }
                labels = []
                if annotations:
                    for i in annotations:
                        start_idx = i["start_offset"]
                        end_idx = i["end_offset"]
                        label = label_maping[i["label"]]
                        labels.append([start_idx, end_idx, label])
            else:
                text = line["text"][4:]
                labels = []
                if line["labels"]:
                    labels = [[l[0] - 4, l[1] - 4, l[2]] for l in line["labels"]]

            tokens = list(text)

            labeled_data_dict = {}
            for idx, char in enumerate(tokens):
                labeled_data_dict[idx] = [char, "O"]

            for label_ in labels:
                start_idx, end_idx, label = label_
                for i in range(start_idx, end_idx):
                    if i == start_idx:
                        labeled_data_dict[i][1] = f"B-{label}"
                    else:
                        labeled_data_dict[i][1] = f"I-{label}"
            labeled_data += labeled_data_dict.values()

    # 分句
    labeled_data_list, labeled_data_seq = [], []

    for data in labeled_data:
        if data[0] == "\n\":
            if labeled_data_seq:
                labeled_data_list.append(labeled_data_seq)
            labeled_data_seq = []
        elif data[0] == " ":
            continue
        else:
            labeled_data_seq.append(data)

    # space

    return labeled_data_list
